# SnapTo3D Spatial‑Planning Pipeline

Flow: 
```
USER prompt -> LLM (scene‑graph JSON) -> Constraint Solver -> Blender CodeGen -> Render -> VLM Verifier
```

## 0  Setup and dependencies

In [38]:
# ↳ Run once per environment
!source spatial-env/bin/activate
# !pip install -r requirements.txt

zsh:source:1: no such file or directory: spatial-env/bin/activate


## 1  Scene‑graph schema & helper classes

In [39]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv

from ortools.sat.python import cp_model
import time

import subprocess
import pathlib
import cloudinary
import cloudinary.uploader

from cloudinary.utils import cloudinary_url
import os
from loguru import logger

load_dotenv()

True

## 2  LLM reasoning → JSON

In [40]:
MODEL = 'gpt-4'

OpenAPIClient = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_to_scene(prompt: str) -> dict:
    function = {
    "name": "create_scene_graph",
    "description": "Creates a comprehensive 3D scene graph with room architecture, furniture, and materials from a description",
    "parameters": {
        "type": "object",
        "properties": {
            "metadata": {
                "type": "object",
                "description": "Scene-level metadata",
                "properties": {
                    "projectId": {"type": "string", "description": "Unique scene/room ID"},
                    "roomType": {"type": "string", "enum": ["bedroom", "lounge", "bathroom", "kitchen", "office", "dining"], "description": "Type of room"},
                    "units": {"type": "string", "description": "Linear unit symbol (e.g., 'm', 'ft')"},
                    "axes": {
                        "type": "object",
                        "properties": {
                            "right": {"type": "string", "description": "Axis pointing right (e.g., 'X+')"},
                            "forward": {"type": "string", "description": "Axis pointing forward (e.g., 'Y+')"},
                            "up": {"type": "string", "description": "Axis pointing up (e.g., 'Z+')"}
                        },
                        "required": ["right", "forward", "up"]
                    }
                },
                "required": ["projectId", "roomType", "units", "axes"]
            },
            "room": {
                "type": "object",
                "description": "Architectural shell definition",
                "properties": {
                    "dims": {
                        "type": "object",
                        "properties": {
                            "w": {"type": "number", "description": "Interior width"},
                            "d": {"type": "number", "description": "Interior depth"},
                            "h": {"type": "number", "description": "Interior height"}
                        },
                        "required": ["w", "d", "h"]
                    },
                    "walls": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "id": {"type": "string", "description": "Stable wall identifier"},
                                "start": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "World-space XYZ of wall start"},
                                "end": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "World-space XYZ of wall end"},
                                "thickness": {"type": "number", "description": "Wall thickness"},
                                "mat": {"type": "string", "description": "Material ID"},
                                "mesh": {"type": ["string", "null"], "description": "Optional custom mesh path"}
                            },
                            "required": ["id", "start", "end", "thickness", "mat"]
                        }
                    },
                    "openings": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "id": {"type": "string", "description": "Door/window identifier"},
                                "wallId": {"type": "string", "description": "Host wall's ID"},
                                "offset": {"type": "number", "description": "Distance from wall start"},
                                "w": {"type": "number", "description": "Opening width"},
                                "h": {"type": "number", "description": "Opening height"},
                                "sill": {"type": "number", "description": "Z-offset of bottom edge"}
                            },
                            "required": ["id", "wallId", "offset", "w", "h", "sill"]
                        }
                    },
                    "floorMat": {"type": "string", "description": "Default floor material ID"}
                },
                "required": ["dims", "walls", "floorMat"]
            },
            "furniture": {
                "type": "array",
                "description": "Furnishings and decor objects",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {"type": "string", "description": "Instance name"},
                        "catalogId": {"type": "string", "description": "SKU/external reference"},
                        "category": {"type": "string", "description": "Furniture category (e.g., bed, sofa, sink)"},
                        "dims": {
                            "type": ["object", "null"],
                            "properties": {
                                "w": {"type": "number"},
                                "d": {"type": "number"},
                                "h": {"type": "number"},
                                "source": {"type": "string", "enum": ["catalog", "inferred", "user"]}
                            }
                        },
                        "xform": {
                            "type": "object",
                            "description": "Blender-ready transform",
                            "properties": {
                                "pos": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "Translation XYZ"},
                                "rot": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "Euler rotation degrees XYZ"},
                                "scale": {"type": "array", "items": {"type": "number"}, "minItems": 3, "maxItems": 3, "description": "Non-uniform scale factors"}
                            },
                            "required": ["pos", "rot", "scale"]
                        },
                        "mesh": {"type": ["string", "null"], "description": "File path to mesh/prefab"},
                        "matOverride": {"type": ["string", "null"], "description": "Optional material swap"},
                        "tags": {"type": "array", "items": {"type": "string"}, "description": "Semantic labels"}
                    },
                    "required": ["id", "catalogId", "category", "xform"]
                }
            },
            "constraints": {
                "type": "array",
                "description": "Declarative layout rules",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {"type": "string", "description": "Human-readable slug"},
                        "type": {"type": "string", "enum": ["offset", "align", "inside", "groupAlign"], "description": "Constraint type"},
                        "targets": {"type": "array", "items": {"type": "string"}, "description": "IDs of objects/walls involved"},
                        "axis": {"type": "string", "enum": ["X", "Y", "Z"], "description": "Axis for offset/align rules"},
                        "distance": {"type": "number", "description": "Gap size"},
                        "relativeTo": {"type": "string", "description": "Anchoring object ID"},
                        "alignAxis": {"type": "string", "description": "Axis to match in groupAlign"}
                    },
                    "required": ["id", "type", "targets"]
                }
            },
            "materials": {
                "type": "array",
                "description": "Shared material palette",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {"type": "string", "description": "Material name"},
                        "color": {"type": "array", "items": {"type": "number", "minimum": 0, "maximum": 1}, "minItems": 3, "maxItems": 3, "description": "RGB 0-1 values"},
                        "tex": {"type": ["string", "null"], "description": "Path to texture map"},
                        "roughness": {"type": ["number", "null"], "minimum": 0, "maximum": 1, "description": "PBR roughness 0-1"},
                        "scaleUV": {"type": ["number", "null"], "description": "Texture tiling factor"}
                    },
                    "required": ["id"]
                }
            }
        },
        "required": ["metadata", "room", "furniture", "materials"]
    }
}
    
    system_message = """
                        You are a 3D spatial planning assistant. 
                        Generate realistic room layouts with proper furniture placement, considering spatial relationships, accessibility, and design principles.
                        Always include complete scene graphs with dimensions, transforms, and materials.
        """
    
    try:
        response = OpenAPIClient.chat.completions.create(
            model=MODEL,
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ],
            functions=[function],
            function_call={'name':'create_scene_graph'}
        )
        
        if not response.choices or not response.choices[0].message.function_call:
            raise ValueError("No valid response from API")
            
        logger.info(f"Scene graph generated successfully ...")
        return json.loads(response.choices[0].message.function_call.arguments)
    except Exception as e:
        logger.error(f"Error in llm_to_scene: {str(e)}")
        raise

## 3  Constraint solver → absolute coordinates

In [41]:
GRID = 100 
MAX_RETRIES = 5

def solve_scene(scene: dict) -> dict:
    """
    Solve scene constraints. Returns placements dict on success, False on error.
    """
    logger.info(f"Solving scene constraints ...")

    try:
        m = cp_model.CpModel()
        pos, half = {}, {}

        if scene.get('constraints') is None:
            logger.error(f"No constraints found in scene")
            return False

        # Extract furniture items and convert to expected format
        for item in scene['furniture']:
            # Handle different dimension formats
            if 'w' in item['dims'] and 'l' in item['dims']:
                w = item['dims']['w']
                d = item['dims']['l']  # 'l' (length) maps to 'd' (depth)
            elif 'balloon' in item['dims']:  # Special case for lamp
                w = item['dims']['balloon']['r'] * 2
                d = item['dims']['balloon']['r'] * 2
            else:
                w, d = 0.5, 0.5 
                
            half[item['id']] = (int(w*GRID/2), int(d*GRID/2))
            pos[item['id']]  = (                     
                m.NewIntVar(0, int(scene['room']['dims']['w']*GRID), f'x_{item["id"]}'),
                m.NewIntVar(0, int(scene['room']['dims']['d']*GRID), f'y_{item["id"]}')
            )

        wall_positions = {}
        for wall in scene['room']['walls']:
            if wall['id'] == 'wall1':  # Bottom wall (Y=0)
                wall_positions[wall['id']] = {'y': 0}
            elif wall['id'] == 'wall2':  # Left wall (X=0)
                wall_positions[wall['id']] = {'x': 0}
            elif wall['id'] == 'wall3':  # Right wall
                wall_positions[wall['id']] = {'x': scene['room']['dims']['w']}
            elif wall['id'] == 'wall4':  # Top wall
                wall_positions[wall['id']] = {'y': scene['room']['dims']['d']}

        # relational constraints --------------------------------------------------
        for c in scene['constraints']:
            if c['type'] == 'offset':
                for target in c['targets']:
                    if target not in pos:
                        logger.error(f"Target '{target}' not found in furniture")
                        return False
                        
                    distance = int(c['distance'] * GRID)
                    x, y = pos[target]
                    w, d = half[target]
                    
                    if c['relativeTo'] not in wall_positions:
                        print(f"Error: Wall '{c['relativeTo']}' not found")
                        return False
                        
                    wall_pos = wall_positions[c['relativeTo']]
                    
                    if c['axis'] == 'X' and 'x' in wall_pos:
                        if wall_pos['x'] == 0:  # Left wall
                            m.Add(x - w == distance)
                        else:  # Right wall
                            m.Add(x + w == int(wall_pos['x'] * GRID) - distance)
                    elif c['axis'] == 'Y' and 'y' in wall_pos:
                        if wall_pos['y'] == 0:  # Bottom wall
                            m.Add(y - d == distance)
                        else:  # Top wall
                            m.Add(y + d == int(wall_pos['y'] * GRID) - distance)
                            
            elif c['type'] == 'align':
                # Handle alignment constraints
                for target in c['targets']:
                    if target not in pos:
                        logger.error(f"Target '{target}' not found in furniture")
                        return False
                    if c['relativeTo'] not in pos:
                        logger.error(f"RelativeTo '{c['relativeTo']}' not found in furniture")
                        return False
                        
                    xa, ya = pos[target]
                    xb, yb = pos[c['relativeTo']]
                    
                    if c.get('alignAxis') == 'Y':
                        # Align along Y axis (same Y coordinate)
                        m.Add(ya == yb)
                    elif c.get('alignAxis') == 'X':
                        # Align along X axis (same X coordinate)
                        m.Add(xa == xb)

        # NON-OVERLAP (boolean disjunction) ---------------------------------------
        ids = list(pos.keys())
        for i in range(len(ids)):
            for j in range(i+1, len(ids)):
                A, B = ids[i], ids[j]
                xa, ya = pos[A]; xb, yb = pos[B]
                wa, da = half[A]; wb, db = half[B]

                if (A == 'lamp1' and B == 'desk1') or (A == 'desk1' and B == 'lamp1'):
                    continue

                left   = m.NewBoolVar(f'{A}_left_{B}')
                right  = m.NewBoolVar(f'{A}_right_{B}')
                front  = m.NewBoolVar(f'{A}_front_{B}')
                behind = m.NewBoolVar(f'{A}_behind_{B}')

                m.Add(xa + wa <= xb - wb).OnlyEnforceIf(left)
                m.Add(xb + wb <= xa - wa).OnlyEnforceIf(right)
                m.Add(ya + da <= yb - db).OnlyEnforceIf(front)
                m.Add(yb + db <= ya - da).OnlyEnforceIf(behind)

                m.AddBoolOr([left, right, front, behind])

        # -------------------------------------------------------------------------
        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 10.0  # Add timeout
        status = solver.Solve(m)
        
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            return {oid: (solver.Value(x)/GRID, solver.Value(y)/GRID)
                    for oid,(x,y) in pos.items()}
        else:
            logger.error(f"Solver failed with status: {solver.StatusName(status)}")
            if status == cp_model.INFEASIBLE:
                logger.error("The constraints are contradictory and cannot be satisfied.")
            return False
            
    except Exception as e:
        logger.error(f"Error in solve_scene: {str(e)}")
        return False

## 4  CodeGen → Blender .py script

In [42]:
def gen_blender_py(scene: dict, placements: dict, outfile='assets/blender/script.py'):
    """
    Generate a standalone Blender Python script from a scene graph plus placements.

    Args:
        scene:       Dictionary with room / walls / furniture / materials
        placements:  Dict mapping furniture IDs → (x, y) positions (metres)
        outfile:     Filepath for the generated .py script
    """
    lines = [
        "import bpy",
        "from mathutils import Vector",
        "import math",
        "",
        "# -------------------------------------------------- Reset scene",
        "bpy.ops.object.select_all(action='SELECT')",
        "bpy.ops.object.delete(use_global=False)",
        "",
    ]

    logger.info("Generating Blender script → %s", outfile)

    # ---- room envelope -----------------------------------------------------
    room_dims  = scene.get('room', {}).get('dims', {'w': 4, 'd': 2, 'h': 3})
    room_w     = room_dims['w']
    room_d     = room_dims['d']
    room_h     = room_dims['h']

    # ---- CAMERA (auto-framing isometric) -----------------------------------
    diag       = max(room_w, room_d)
    cam_loc    = (room_w / 2 + diag * 1.20,
                  -diag * 1.20,
                  room_h + diag * 0.90)
    cam_target = (room_w / 2, room_d / 2, room_h / 2)

    lines += [
        "# -------------------------------------------------- Camera",
        "cam_data  = bpy.data.cameras.new(name='Camera')",
        "cam_data.lens = 35",
        "cam_obj   = bpy.data.objects.new('Camera', cam_data)",
        "bpy.context.collection.objects.link(cam_obj)",
        f"cam_obj.location = {cam_loc}",
        "",
        "# Aim camera at centre of room (65° elevation, 45° azimuth iso-view)",
        f"direction = Vector({cam_target}) - cam_obj.location",
        "cam_obj.rotation_euler = direction.to_track_quat('-Z', 'Y').to_euler()",
        "bpy.context.scene.camera = cam_obj",
        "",
    ]

    # ---- LIGHT -------------------------------------------------------------
    lines += [
        "# -------------------------------------------------- Lighting",
        "bpy.ops.object.light_add(type='SUN', location=(5, -5, 10))",
        "bpy.context.active_object.data.energy = 2.0",
        "",
    ]

    # ---- RENDER SETTINGS ---------------------------------------------------
    lines += [
        "# -------------------------------------------------- Render settings",
        "scene = bpy.context.scene",
        "scene.render.engine = 'CYCLES'",
        "scene.cycles.samples = 64",
        "scene.render.resolution_x = 1920",
        "scene.render.resolution_y = 1080",
        "",
    ]

    # ---- FLOOR -------------------------------------------------------------
    lines += [
        "# -------------------------------------------------- Floor",
        f"bpy.ops.mesh.primitive_cube_add(location=({room_w/2}, {room_d/2}, -0.05))",
        "floor = bpy.context.active_object",
        "floor.name = 'Floor'",
        f"floor.scale = ({room_w/2}, {room_d/2}, 0.05)",
        "",
    ]

    # ---- WALLS (optional) ---------------------------------------------------
    if 'walls' in scene.get('room', {}):
        lines.append("# -------------------------------------------------- Walls")
        for i, wall in enumerate(scene['room']['walls']):
            wall_id   = wall.get('id', f'wall_{i+1}')
            start, end = wall['start'], wall['end']
            thick      = wall.get('thickness', 0.2)

            if start[0] != end[0]:        # horizontal wall (X axis length)
                cx = (start[0] + end[0]) / 2
                cy = start[1]
                cz = room_h / 2
                sx = abs(end[0] - start[0]) / 2
                sy = thick / 2
                sz = room_h / 2
            else:                         # vertical wall (Y axis length)
                cx = start[0]
                cy = (start[1] + end[1]) / 2
                cz = room_h / 2
                sx = thick / 2
                sy = abs(end[1] - start[1]) / 2
                sz = room_h / 2

            lines += [
                f"bpy.ops.mesh.primitive_cube_add(location=({cx}, {cy}, {cz}))",
                "wall = bpy.context.active_object",
                f"wall.name = '{wall_id}'",
                f"wall.scale = ({sx}, {sy}, {sz})",
                "",
            ]

    # ---- FURNITURE ---------------------------------------------------------
    lines.append("# -------------------------------------------------- Furniture")
    for item in scene.get('furniture', []):
        fid  = item['id']
        x, y = placements.get(fid, (item.get('xform', {}).get('pos', [1, 0, 1])[:2]))
        dims = item.get('dims', {})
        w, d, h = dims.get('w', 1.0), dims.get('d', dims.get('l', 1.0)), dims.get('h', 0.5)
        z   = h / 2
        rot = item.get('xform', {}).get('rot', [0, 0, 0])
        cat = item.get('category', '').lower()

        if 'lamp' in cat:
            primitive = "primitive_cylinder_add"
            scale     = f"({w/2:.2f}, {d/2:.2f}, {h/2:.2f})"
        else:
            primitive = "primitive_cube_add"
            scale     = f"({w/2:.2f}, {d/2:.2f}, {h/2:.2f})"

        lines += [
            f"# {fid}  ({cat})",
            f"bpy.ops.mesh.{primitive}(location=({x:.2f}, {y:.2f}, {z:.2f}))",
            "obj = bpy.context.active_object",
            f"obj.name = '{fid}'",
            f"obj.scale = {scale}",
            f"obj.rotation_euler = ({rot[0]}, {rot[1]}, {rot[2]})",
            "",
        ]

    # ---- MATERIALS ---------------------------------------------------------
    if 'materials' in scene:
        lines.append("# -------------------------------------------------- Materials")
        for mat in scene['materials']:
            mid, col = mat['id'], mat.get('color', [200, 200, 200])
            rough    = mat.get('roughness', 0.5)
            r, g, b  = [c/255 for c in col]
            lines += [
                f"mat = bpy.data.materials.new(name='{mid}')",
                "mat.use_nodes = True",
                "bsdf = mat.node_tree.nodes['Principled BSDF']",
                f"bsdf.inputs['Base Color'].default_value = ({r:.3f}, {g:.3f}, {b:.3f}, 1.0)",
                f"bsdf.inputs['Roughness'].default_value = {rough}",
                "",
            ]

        # auto-assign floor / wall mats if specified
        if 'floorMat' in scene.get('room', {}):
            fm = scene['room']['floorMat']
            lines += [
                "floor = bpy.data.objects.get('Floor')",
                f"if floor and '{fm}' in bpy.data.materials:",
                f"    floor.data.materials.append(bpy.data.materials['{fm}'])",
                "",
            ]
        for wall in scene.get('room', {}).get('walls', []):
            if 'mat' in wall:
                mid = wall['mat']
                wid = wall['id']
                lines += [
                    f"w = bpy.data.objects.get('{wid}')",
                    f"if w and '{mid}' in bpy.data.materials:",
                    f"    w.data.materials.append(bpy.data.materials['{mid}'])",
                    "",
                ]

    # ---- SAVE + RENDER -----------------------------------------------------
    lines += [
        "# -------------------------------------------------- Save & Render",
        "import os",
        "os.makedirs('assets/renders', exist_ok=True)",
        "bpy.ops.wm.save_as_mainfile(filepath='assets/renders/scene.blend')",
        "",
        "bpy.context.scene.render.filepath = 'assets/renders/scene.png'",
        "bpy.ops.render.render(write_still=True)",
    ]

    logger.info("…%d lines of Blender script generated", len(lines))

    with open(outfile, 'w') as f:
        f.write('\n'.join(lines))

    return outfile

## 5  Headless Blender render

In [43]:
cloudinary.config(
    cloud_name=os.getenv('CLOUDINARY_CLOUD_NAME'),
    api_key=os.getenv('CLOUDINARY_API_KEY'),
    api_secret=os.getenv('CLOUDINARY_API_SECRET')
)

def render_and_upload_image(pyfile: str, image_outfile: str = 'assets/renders/scene.png',
                           cloudinary_folder: str = 'blender_renders'):
    """
    Uploading rendered images (PNG, JPG, etc.)
    
    Args:
        pyfile: Path to the Python script for Blender
        image_outfile: Output path for the rendered image
        cloudinary_folder: Folder name in Cloudinary
    
    Returns:
        dict: Cloudinary upload response
    """
    logger.info(f"Rendering and uploading image to Cloudinary ...")
    blender_bin = '/Applications/Blender.app/Contents/MacOS/Blender'
    
    output_path = pathlib.Path(image_outfile)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    cmd = [blender_bin, '-b', '--python', pyfile]
    logger.info(f"Running Blender: {' '.join(cmd)}")
    
    try:
        subprocess.run(cmd, check=True)
        logger.success(f"Blender rendering completed: {output_path}")
    except subprocess.CalledProcessError as e:
        logger.error(f"Blender rendering failed: {e}")
        raise
    
    try:
        logger.info(f"Uploading {output_path} to Cloudinary...")
        
        upload_result = cloudinary.uploader.upload(
            str(output_path),
            folder=cloudinary_folder,
            public_id=f"{output_path.stem}_{int(time.time())}",
            overwrite=True,
            tags=["blender", "render", "image"],
        )
        
        logger.success(f"Upload successfull {upload_result['public_id']} !!")

        return upload_result
        
    except Exception as e:
        logger.error(f"Cloudinary upload failed: {e}")
        raise

## 6  Vision‑language verifier (stub)

In [ ]:
def verify_scene(image_path: str, scene: str) -> float:
    """Return fraction of relations judged correct by GPT-4V (stub)."""

    logger.info(f"Verifying scene ...") 

    system_message = 'You are an expert in evaluating PIL image objects, which contain rooms generated in Blender, and assigning them a score based on their spatial accuracy.'
    prompt = f"Ascribe a decimal number ranging from 0.0-1.0 that scores the image in the following link {image_path} based on how well the generated scene satisfies the following JSON layout: {scene}"


    response = OpenAPIClient.chat.completions.create(
            model='o4-mini',
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ],
    )

    return response.choices[0].message.content # assume perfect


In [ ]:
def generate_questions(image_path: str, scene: str, test: str) -> float:
    """Return evaluation questions given a room scene using GPT-4V (stub)."""

    if test == "stress":
        logger.info(f"Generating question ...") 

        system_message = 'You are an expert in asking questions to evaluate the spatial accuracy of a generated room with a large amount of furniture. You do this by examining whether or not the furniture in the image overlaps.' # many furnitures will overlap
        prompt = f"Generate a YES or NO questions that evaluates an image of a generated blender which can be found in the following link: {image_path}. Use specific furniture items specified in the following JSON layout: {scene}."


        response = OpenAPIClient.chat.completions.create(
                model=MODEL,
                messages=[
                    {'role':'system', 'content': system_message},
                    {'role':'user', 'content': prompt}
                ],
        )
        return response.choices[0].message.content # assume perfect
    
    return None

In [46]:
def answer_questions(image_path: str, question: str) -> float:
    """Return an answer on an evaluation question given a room scene."""

    logger.info(f"Generating answer ...") 

    system_message = 'You are an expert in answering questions that evaluate the spatial accuracy of a given room with large amounts of furniture.' # many furnitures will overlap
    prompt = f"Generate an answer to the following question: {question} given the room image in the following link: {image_path}"


    response = OpenAPIClient.chat.completions.create(
            model="o4-mini",
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ],
    )
    return response.choices[0].message.content # assume perfect

In [47]:
def save_blender_code(code, filename="blender_scene.py"):
    """Save the generated Blender Python code to a file"""
    logger.info(f"Saving Blender Python code to {filename} ...")
    # Create directory if it doesn't exist
    os.makedirs("blender_scripts", exist_ok=True)
    
    # Full path to save the file
    filepath = os.path.join("blender_scripts", filename)
    
    # Write the code to file
    with open(filepath, "w") as f:
        f.write(code)
    
    logger.success(f"Blender Python code saved to: {filepath}")

    return filepath

## 7  End‑to‑end pipeline

In [48]:
def plan_and_render(prompt: str):
    """
    Plan and render with automatic retry on constraint failures.
    After 2 attempts, continues with original positions if constraints fail.
    
    Args:
        prompt: Initial prompt for scene generation
        llm_to_scene: Function to convert prompt to scene JSON
        gen_blender_py: Function to generate Blender Python script
        render_blend: Function to render the Blender file
        verify_scene: Optional function to verify the scene
        eval_question1, eval_question2: Questions that verify spatial accuracy of image. Varies based on type of test
    """

    logger.info(f"Planning and rendering ...")
    
    attempt = 0
    current_prompt = prompt
    last_error_msg = ""
    last_scene = None
    CONSTRAINT_RETRY_LIMIT = 2  # After this many tries, use original positions
    
    while attempt < MAX_RETRIES:
        attempt += 1
        logger.info(f"Attempt {attempt}/{MAX_RETRIES}")
        
        try:
            logger.info("Generating scene from prompt ...")
            scene = llm_to_scene(current_prompt)
            last_scene = scene  # Save the last generated scene
            logger.success('Scene generated successfully.')
            
            logger.info("Solving constraints ...")
            placements = solve_scene(scene)
            
            if placements is False:
                logger.error(f"Constraint solving failed on attempt {attempt}")
                
                if attempt < CONSTRAINT_RETRY_LIMIT:
                    # Try again with modified prompt
                    last_error_msg = "Previous attempt failed due to unsatisfiable constraints. "
                    current_prompt = f"{prompt}\n\nIMPORTANT: {last_error_msg}Please ensure furniture can fit in the room with proper spacing and avoid conflicting constraints."
                    
                    logger.info("Waiting before retry...")
                    time.sleep(1)  # Brief pause before retry
                    continue
                else:
                    # After 2 tries, just use original positions from the scene
                    logger.warning(f"Constraint solving failed after {CONSTRAINT_RETRY_LIMIT} attempts.")
                
                    
                    # Extract original positions from the scene
                    placements = {}
                    for item in scene.get('furniture', []):
                        if 'id' in item and 'xform' in item and 'pos' in item['xform']:
                            # Use X,Y coordinates from original position
                            placements[item['id']] = (item['xform']['pos'][0], item['xform']['pos'][1])
                        else:
                            # Default position if not specified
                            placements[item['id']] = (1.0, 1.0)
                    
                    logger.info(f"Using original placements: {placements}")
                    
            
            logger.success(f"\n✓ Proceeding with placements: {placements}")
            
            logger.info("\nGenerating Blender script...")
            try:
                py_script = gen_blender_py(scene, placements) 
                logger.success("Blender script generated successfully.")
            except Exception as e:
                logger.error(f"Error generating Blender script: {e}")
                
                if attempt < MAX_RETRIES:
                    current_prompt = f"{prompt}\n\nNote: Blender script generation failed. Please ensure valid scene structure."
                    continue
                else:
                    return None
        except Exception as e:
            logger.error(f"\nUnexpected error: {str(e)}")

            if attempt < MAX_RETRIES:
                current_prompt = f"{prompt}\n\nNote: Previous attempt failed with error: {str(e)}. Please try a different approach."
                time.sleep(1)
                continue
            else:
                logger.error(f"Failed after {MAX_RETRIES} attempts.")
                
                return None
                
    logger.info("Rendering scene...")
    try:
        rendered_scene =  render_and_upload_image(pyfile="assets/blender/script.py", image_outfile="assets/renders/scene.png")
        if not rendered_scene:
            raise ValueError("Render function returned empty result")
            
    except Exception as e:
        logger.error(f"Error during rendering: {e}")

        if attempt < MAX_RETRIES:
            current_prompt = f"{prompt}\n\nNote: Rendering failed with error: {str(e)}. Please try a simpler scene."
        else:
            return None

    logger.info("Verifying scene ...")
    score = verify_scene(rendered_scene['secure_url'], scene)
    logger.success(f'Verification score: {score}')

    eval_question1 = generate_questions(rendered_scene['secure_url'], scene=scene, test="stress")
    eval_answer1 = answer_questions(rendered_scene['secure_url'], eval_question1)
    logger.success(f'Question 1: {eval_question1}')
    logger.success(f'Answer 1: {eval_answer1}')


    eval_question2 = generate_questions(rendered_scene['secure_url'], scene=scene, test="stress")
    eval_answer2 = answer_questions(rendered_scene['secure_url'], eval_question2)
    logger.success(f'Question 2: {eval_question2}')
    logger.success(f'Answer 2: {eval_answer2}')

    logger.success(f"\nSuccess! Scene rendered successfully.")
    return score, eval_question1, eval_answer1, eval_question2, eval_answer2, rendered_scene


## 8  Example run

In [49]:
# Bedroom
bedroom_prompt = "Create a bedroom with dimensions 4 x 2 x 3 meters. Include 3 beds, 3 desks, 3 lamps, a standing bookshelf, and a plant. Position furniture logically with proper spacing and accessibility."
bed_score, bed_eval_question1, bed_eval_answer1, bed_eval_question2, bed_eval_answer2, bed_rendered_scene = plan_and_render(bedroom_prompt)

2025-06-02 21:48:04.875 | INFO     | __main__:plan_and_render:15 - Planning and rendering ...
2025-06-02 21:48:04.875 | INFO     | __main__:plan_and_render:25 - Attempt 1/5
2025-06-02 21:48:04.876 | INFO     | __main__:plan_and_render:28 - Generating scene from prompt ...
2025-06-02 21:49:07.069 | INFO     | __main__:llm_to_scene:170 - Scene graph generated successfully ...
2025-06-02 21:49:07.073 | SUCCESS  | __main__:plan_and_render:31 - Scene generated successfully.
2025-06-02 21:49:07.074 | INFO     | __main__:plan_and_render:33 - Solving constraints ...
2025-06-02 21:49:07.075 | INFO     | __main__:solve_scene:8 - Solving scene constraints ...
2025-06-02 21:49:07.077 | ERROR    | __main__:solve_scene:15 - No constraints found in scene
2025-06-02 21:49:07.077 | ERROR    | __main__:plan_and_render:37 - Constraint solving failed on attempt 1
2025-06-02 21:49:07.077 | INFO     | __main__:plan_and_render:44 - Waiting before retry...
2025-06-02 21:49:08.083 | INFO     | __main__:plan_an

Blender 4.4.3 (hash 802179c51ccc built 2025-04-29 14:25:11)
Fra:1 Mem:10.92M (Peak 10.92M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | Floor
Fra:1 Mem:10.92M (Peak 10.92M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall1
Fra:1 Mem:10.92M (Peak 10.92M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall3
Fra:1 Mem:10.92M (Peak 10.92M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall4
Fra:1 Mem:10.92M (Peak 10.92M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall2
Fra:1 Mem:10.92M (Peak 10.92M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | bed1
Fra:1 Mem:10.92M (Peak 10.92M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | plant
Fra:1 Mem:10.92M (Peak 10.92M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Sy

2025-06-02 21:54:38.627 | SUCCESS  | __main__:render_and_upload_image:31 - Blender rendering completed: assets/renders/scene.png
2025-06-02 21:54:38.629 | INFO     | __main__:render_and_upload_image:37 - Uploading assets/renders/scene.png to Cloudinary...



Blender quit


2025-06-02 21:54:39.316 | SUCCESS  | __main__:render_and_upload_image:47 - Upload successfull blender_renders/scene_1748926478 !!
2025-06-02 21:54:39.318 | INFO     | __main__:plan_and_render:105 - Verifying scene ...
2025-06-02 21:54:39.319 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-02 21:54:48.872 | SUCCESS  | __main__:plan_and_render:107 - Verification score: 0.20
2025-06-02 21:54:48.873 | INFO     | __main__:generate_questions:5 - Generating question ...
2025-06-02 21:54:50.365 | INFO     | __main__:answer_questions:4 - Generating answer ...
2025-06-02 21:54:54.705 | SUCCESS  | __main__:plan_and_render:111 - Question 1: Is the bed (bed1) overlapping with the bookshelf in the image?
2025-06-02 21:54:54.707 | SUCCESS  | __main__:plan_and_render:112 - Answer 1: No. In that layout bed1 sits on the right side of the room against the far wall, while the bookshelf is mounted on the opposite (left) wall near the windows, with clear floor space and furniture in betwe

In [55]:
bed_rendered_scene

{'asset_id': '771458a5357618fd148dd21c2654372c',
 'public_id': 'blender_renders/scene_1748926478',
 'version': 1748926478,
 'version_id': '6ff49f9ee5850e2110167ca44ee76d06',
 'signature': '326f7e074fc2aaea810d45a9b4f61db8ccf9c7c9',
 'width': 1920,
 'height': 1080,
 'format': 'png',
 'resource_type': 'image',
 'created_at': '2025-06-03T04:54:38Z',
 'tags': ['blender', 'render', 'image'],
 'bytes': 1344719,
 'type': 'upload',
 'etag': 'f62ead46c66037ea6906c857445f49d1',
 'placeholder': False,
 'url': 'http://res.cloudinary.com/cld-store/image/upload/v1748926478/blender_renders/scene_1748926478.png',
 'secure_url': 'https://res.cloudinary.com/cld-store/image/upload/v1748926478/blender_renders/scene_1748926478.png',
 'folder': 'blender_renders',
 'original_filename': 'scene',
 'api_key': '986284565458351'}

In [50]:
# Bathroom
bathroom_prompt = "Create a bathroom with dimensions 4 x 2 x 3 meters. Include 3 toilets, 3 bathtubs, 3 sinks, a mirror, and a shower. Position furniture logically with proper spacing and accessibility."
bath_score, bath_eval_question1, bath_eval_answer1, bath_eval_question2, bath_eval_answer2, bath_rendered_scene = plan_and_render(bathroom_prompt)

2025-06-02 21:54:59.538 | INFO     | __main__:plan_and_render:15 - Planning and rendering ...
2025-06-02 21:54:59.540 | INFO     | __main__:plan_and_render:25 - Attempt 1/5
2025-06-02 21:54:59.541 | INFO     | __main__:plan_and_render:28 - Generating scene from prompt ...
2025-06-02 21:56:14.483 | INFO     | __main__:llm_to_scene:170 - Scene graph generated successfully ...
2025-06-02 21:56:14.486 | SUCCESS  | __main__:plan_and_render:31 - Scene generated successfully.
2025-06-02 21:56:14.486 | INFO     | __main__:plan_and_render:33 - Solving constraints ...
2025-06-02 21:56:14.487 | INFO     | __main__:solve_scene:8 - Solving scene constraints ...
2025-06-02 21:56:14.543 | SUCCESS  | __main__:plan_and_render:65 - 
✓ Proceeding with placements: {'toilet1': (0.0, 0.0), 'toilet2': (0.0, 0.5), 'toilet3': (0.0, 1.0), 'bathtub1': (0.5, 0.0), 'bathtub2': (1.0, 0.0), 'bathtub3': (1.5, 0.0), 'sink1': (2.0, 0.0), 'sink2': (2.5, 0.0), 'sink3': (3.0, 0.0), 'mirror': (3.5, 0.0), 'shower': (4.0, 0.

Blender 4.4.3 (hash 802179c51ccc built 2025-04-29 14:25:11)
Fra:1 Mem:11.33M (Peak 11.33M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | Floor
Fra:1 Mem:11.33M (Peak 11.33M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall1
Fra:1 Mem:11.33M (Peak 11.33M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall4
Fra:1 Mem:11.33M (Peak 11.33M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | toilet1
Fra:1 Mem:11.33M (Peak 11.33M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | toilet2
Fra:1 Mem:11.33M (Peak 11.33M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | shower1
Fra:1 Mem:11.34M (Peak 11.34M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | toilet3
Fra:1 Mem:11.34M (Peak 11.34M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewL

2025-06-02 22:01:39.312 | SUCCESS  | __main__:render_and_upload_image:31 - Blender rendering completed: assets/renders/scene.png
2025-06-02 22:01:39.313 | INFO     | __main__:render_and_upload_image:37 - Uploading assets/renders/scene.png to Cloudinary...


Info: Saved "scene.blend"

Blender quit


2025-06-02 22:01:40.724 | SUCCESS  | __main__:render_and_upload_image:47 - Upload successfull blender_renders/scene_1748926899 !!
2025-06-02 22:01:40.725 | INFO     | __main__:plan_and_render:105 - Verifying scene ...
2025-06-02 22:01:40.726 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-02 22:01:44.852 | SUCCESS  | __main__:plan_and_render:107 - Verification score: 0.18
2025-06-02 22:01:44.853 | INFO     | __main__:generate_questions:5 - Generating question ...
2025-06-02 22:01:48.176 | INFO     | __main__:answer_questions:4 - Generating answer ...
2025-06-02 22:01:51.386 | SUCCESS  | __main__:plan_and_render:111 - Question 1: Is there any overlap between 'modern_toilet1' at position [0.7, 0, 0.8] and 'modern_sink1' at position [0.7, 1.9, 0.8] in the image?
2025-06-02 22:01:51.389 | SUCCESS  | __main__:plan_and_render:112 - Answer 1: No—they don’t overlap. Both are centered at x=0.7 m and z=0.8 m (so they line up in width and height), but the sink is at y=1.9 m whi

In [57]:
bath_rendered_scene

{'asset_id': 'b98226c494f7c36cf63fcf1226516796',
 'public_id': 'blender_renders/scene_1748926899',
 'version': 1748926899,
 'version_id': '6b3c4a76bb89253c4980b3f2e6ce56d4',
 'signature': '40fb4ac5aee2775a77b4202c9680f795451e714a',
 'width': 1920,
 'height': 1080,
 'format': 'png',
 'resource_type': 'image',
 'created_at': '2025-06-03T05:01:39Z',
 'tags': ['blender', 'render', 'image'],
 'bytes': 1368981,
 'type': 'upload',
 'etag': '11b6beb0fd0154ac3da4582846c65299',
 'placeholder': False,
 'url': 'http://res.cloudinary.com/cld-store/image/upload/v1748926899/blender_renders/scene_1748926899.png',
 'secure_url': 'https://res.cloudinary.com/cld-store/image/upload/v1748926899/blender_renders/scene_1748926899.png',
 'folder': 'blender_renders',
 'original_filename': 'scene',
 'api_key': '986284565458351'}

In [51]:
# Dining room
dining_room_prompt = "Create a dining room with dimensions 4 x 2 x 3 meters. Include 3 tables, 3 chairs, 3 cabinents, and 2 standing lamps. Position furniture logically with proper spacing and accessibility."
dining_score, dining_eval_question1, dining_eval_answer1, dining_eval_question2, dining_eval_answer2, dining_rendered_scene = plan_and_render(dining_room_prompt)

2025-06-02 22:02:01.780 | INFO     | __main__:plan_and_render:15 - Planning and rendering ...
2025-06-02 22:02:01.782 | INFO     | __main__:plan_and_render:25 - Attempt 1/5
2025-06-02 22:02:01.783 | INFO     | __main__:plan_and_render:28 - Generating scene from prompt ...
2025-06-02 22:03:09.924 | INFO     | __main__:llm_to_scene:170 - Scene graph generated successfully ...
2025-06-02 22:03:09.929 | SUCCESS  | __main__:plan_and_render:31 - Scene generated successfully.
2025-06-02 22:03:09.929 | INFO     | __main__:plan_and_render:33 - Solving constraints ...
2025-06-02 22:03:09.930 | INFO     | __main__:solve_scene:8 - Solving scene constraints ...
2025-06-02 22:03:09.932 | ERROR    | __main__:solve_scene:15 - No constraints found in scene
2025-06-02 22:03:09.933 | ERROR    | __main__:plan_and_render:37 - Constraint solving failed on attempt 1
2025-06-02 22:03:09.933 | INFO     | __main__:plan_and_render:44 - Waiting before retry...
2025-06-02 22:03:10.939 | INFO     | __main__:plan_an

Blender 4.4.3 (hash 802179c51ccc built 2025-04-29 14:25:11)
Fra:1 Mem:10.85M (Peak 10.85M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | Floor
Fra:1 Mem:10.85M (Peak 10.85M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall1
Fra:1 Mem:10.85M (Peak 10.85M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall2
Fra:1 Mem:10.85M (Peak 10.85M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | table1
Fra:1 Mem:10.85M (Peak 10.85M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall4
Fra:1 Mem:10.85M (Peak 10.85M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | chair1
Fra:1 Mem:10.86M (Peak 10.86M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | cabinet1
Fra:1 Mem:10.86M (Peak 10.86M) | Time:00:00.01 | Mem:0.00M, Peak:0.00M | Scene, ViewLaye

2025-06-02 22:07:13.678 | SUCCESS  | __main__:render_and_upload_image:31 - Blender rendering completed: assets/renders/scene.png
2025-06-02 22:07:13.679 | INFO     | __main__:render_and_upload_image:37 - Uploading assets/renders/scene.png to Cloudinary...


Info: Saved "scene.blend"

Blender quit


2025-06-02 22:07:15.000 | SUCCESS  | __main__:render_and_upload_image:47 - Upload successfull blender_renders/scene_1748927233 !!
2025-06-02 22:07:15.001 | INFO     | __main__:plan_and_render:105 - Verifying scene ...
2025-06-02 22:07:15.002 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-02 22:07:21.241 | SUCCESS  | __main__:plan_and_render:107 - Verification score: I’d give this scene a 0.90.

Justification:  
• Room geometry (4 × 2 m footprint, 3 m height) and wall thickness appear correct, with clean right‐angle corners and uniformly white walls.  
• The dark wood floor material matches the specified “wood_dark” in color and finish.  
• The table is centered roughly at (1, 0, 0.8), with correct proportions (0.8 m square, 0.75 m high).  
• The single chair sits behind the table at about (1, 0, 1.3) and is rotated ~180° as required.  
• The cabinet sits close to the rear wall at (0.4, 0, 1.8) with the right orientation, and the standing lamp is near (2, 0, 0.15).  

In [58]:
dining_rendered_scene

{'asset_id': '8f43a121bfc62377526ee8ced7b9dbe5',
 'public_id': 'blender_renders/scene_1748927233',
 'version': 1748927234,
 'version_id': 'f567ed48626dd0df0ccd6e32f4ed62d6',
 'signature': '4b47384c9ad5a3d4619993074919e9d17311b379',
 'width': 1920,
 'height': 1080,
 'format': 'png',
 'resource_type': 'image',
 'created_at': '2025-06-03T05:07:14Z',
 'tags': ['blender', 'render', 'image'],
 'bytes': 1423536,
 'type': 'upload',
 'etag': '5eaf85c61f290bc9731a2d2ac0eedbe0',
 'placeholder': False,
 'url': 'http://res.cloudinary.com/cld-store/image/upload/v1748927234/blender_renders/scene_1748927233.png',
 'secure_url': 'https://res.cloudinary.com/cld-store/image/upload/v1748927234/blender_renders/scene_1748927233.png',
 'folder': 'blender_renders',
 'original_filename': 'scene',
 'api_key': '986284565458351'}

In [52]:
# Living room
living_room_prompt = "Create a living room with dimensions 4 x 2 x 3 meters. Include 3 sofas, 3 couches, 3 lamps, a standing bookshelf, and a plant. Position furniture logically with proper spacing and accessibility."
living_score, living_eval_question1, living_eval_answer1, living_eval_question2, living_eval_answer2, living_rendered_scene = plan_and_render(living_room_prompt)

2025-06-02 22:07:39.546 | INFO     | __main__:plan_and_render:15 - Planning and rendering ...
2025-06-02 22:07:39.547 | INFO     | __main__:plan_and_render:25 - Attempt 1/5
2025-06-02 22:07:39.547 | INFO     | __main__:plan_and_render:28 - Generating scene from prompt ...
2025-06-02 22:08:42.993 | INFO     | __main__:llm_to_scene:170 - Scene graph generated successfully ...
2025-06-02 22:08:42.997 | SUCCESS  | __main__:plan_and_render:31 - Scene generated successfully.
2025-06-02 22:08:42.998 | INFO     | __main__:plan_and_render:33 - Solving constraints ...
2025-06-02 22:08:42.999 | INFO     | __main__:solve_scene:8 - Solving scene constraints ...
2025-06-02 22:08:43.001 | ERROR    | __main__:solve_scene:15 - No constraints found in scene
2025-06-02 22:08:43.001 | ERROR    | __main__:plan_and_render:37 - Constraint solving failed on attempt 1
2025-06-02 22:08:43.002 | INFO     | __main__:plan_and_render:44 - Waiting before retry...
2025-06-02 22:08:44.008 | INFO     | __main__:plan_an

Error: Wall 'Wall1' not found


2025-06-02 22:12:54.381 | INFO     | __main__:llm_to_scene:170 - Scene graph generated successfully ...
2025-06-02 22:12:54.387 | SUCCESS  | __main__:plan_and_render:31 - Scene generated successfully.
2025-06-02 22:12:54.388 | INFO     | __main__:plan_and_render:33 - Solving constraints ...
2025-06-02 22:12:54.389 | INFO     | __main__:solve_scene:8 - Solving scene constraints ...
2025-06-02 22:12:54.445 | SUCCESS  | __main__:plan_and_render:65 - 
✓ Proceeding with placements: {'sofa1': (0.5, 0.0), 'sofa2': (0.0, 0.0), 'sofa3': (1.0, 0.0), 'lamp1': (1.5, 0.0), 'lamp2': (2.0, 0.0), 'lamp3': (2.5, 0.0), 'bookshelf': (3.0, 0.0), 'plant': (3.5, 0.0)}
2025-06-02 22:12:54.446 | INFO     | __main__:plan_and_render:67 - 
Generating Blender script...
2025-06-02 22:12:54.446 | INFO     | __main__:gen_blender_py:21 - Generating Blender script → %s
2025-06-02 22:12:54.446 | INFO     | __main__:gen_blender_py:186 - …%d lines of Blender script generated
2025-06-02 22:12:54.448 | SUCCESS  | __main__:

Blender 4.4.3 (hash 802179c51ccc built 2025-04-29 14:25:11)

Blender quit


2025-06-02 22:17:45.170 | SUCCESS  | __main__:render_and_upload_image:47 - Upload successfull blender_renders/scene_1748927863 !!
2025-06-02 22:17:45.172 | INFO     | __main__:plan_and_render:105 - Verifying scene ...
2025-06-02 22:17:45.172 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-02 22:17:58.295 | SUCCESS  | __main__:plan_and_render:107 - Verification score: 0.85
2025-06-02 22:17:58.296 | INFO     | __main__:generate_questions:5 - Generating question ...
2025-06-02 22:17:59.908 | INFO     | __main__:answer_questions:4 - Generating answer ...
2025-06-02 22:18:05.559 | SUCCESS  | __main__:plan_and_render:111 - Question 1: "Does the second sofa (sofa2) overlap with the second lamp (lamp2) in the image?"
2025-06-02 22:18:05.560 | SUCCESS  | __main__:plan_and_render:112 - Answer 1: No. Sofa 2 and lamp 2 occupy completely different parts of the scene, so their regions do not overlap. In pixel terms, sofa 2 ends around x≈650, while lamp 2 begins at x≈950, so there’

In [56]:
living_rendered_scene

{'asset_id': '2057e2c891316d300ac1cc4f9f80c640',
 'public_id': 'blender_renders/scene_1748927863',
 'version': 1748927864,
 'version_id': '0bd83149152adbd0be1330ccfafe0288',
 'signature': '3039f5e2ff2f36cd7acf9b9022f009235c7d4c2a',
 'width': 1920,
 'height': 1080,
 'format': 'png',
 'resource_type': 'image',
 'created_at': '2025-06-03T05:17:44Z',
 'tags': ['blender', 'render', 'image'],
 'bytes': 1423536,
 'type': 'upload',
 'etag': '5eaf85c61f290bc9731a2d2ac0eedbe0',
 'placeholder': False,
 'url': 'http://res.cloudinary.com/cld-store/image/upload/v1748927864/blender_renders/scene_1748927863.png',
 'secure_url': 'https://res.cloudinary.com/cld-store/image/upload/v1748927864/blender_renders/scene_1748927863.png',
 'folder': 'blender_renders',
 'original_filename': 'scene',
 'api_key': '986284565458351'}

In [53]:
# Office room
office_room_prompt = "Create an office with dimensions 4 x 2 x 3 meters. Include 3 desks, 3 chairs, 3 lamps, 3 bookshelfs, and a sofa. Position furniture logically with proper spacing and accessibility."
office_score, office_eval_question1, office_eval_answer1, office_eval_question2, office_eval_answer2, office_rendered_scene = plan_and_render(office_room_prompt)

2025-06-02 22:18:15.757 | INFO     | __main__:plan_and_render:15 - Planning and rendering ...
2025-06-02 22:18:15.758 | INFO     | __main__:plan_and_render:25 - Attempt 1/5
2025-06-02 22:18:15.759 | INFO     | __main__:plan_and_render:28 - Generating scene from prompt ...
2025-06-02 22:21:05.406 | INFO     | __main__:llm_to_scene:170 - Scene graph generated successfully ...
2025-06-02 22:21:05.416 | SUCCESS  | __main__:plan_and_render:31 - Scene generated successfully.
2025-06-02 22:21:05.417 | INFO     | __main__:plan_and_render:33 - Solving constraints ...
2025-06-02 22:21:05.417 | INFO     | __main__:solve_scene:8 - Solving scene constraints ...
2025-06-02 22:21:05.419 | ERROR    | __main__:solve_scene:15 - No constraints found in scene
2025-06-02 22:21:05.420 | ERROR    | __main__:plan_and_render:37 - Constraint solving failed on attempt 1
2025-06-02 22:21:05.420 | INFO     | __main__:plan_and_render:44 - Waiting before retry...
2025-06-02 22:21:06.425 | INFO     | __main__:plan_an

Blender 4.4.3 (hash 802179c51ccc built 2025-04-29 14:25:11)
Fra:1 Mem:11.47M (Peak 11.47M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | Floor
Fra:1 Mem:11.47M (Peak 11.47M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall1
Fra:1 Mem:11.47M (Peak 11.47M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall2
Fra:1 Mem:11.47M (Peak 11.47M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall3
Fra:1 Mem:11.48M (Peak 11.48M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | wall4
Fra:1 Mem:11.48M (Peak 11.48M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | desk1
Fra:1 Mem:11.48M (Peak 11.48M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | chair1
Fra:1 Mem:11.48M (Peak 11.48M) | Time:00:00.00 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | 

2025-06-02 22:27:18.065 | SUCCESS  | __main__:render_and_upload_image:31 - Blender rendering completed: assets/renders/scene.png
2025-06-02 22:27:18.066 | INFO     | __main__:render_and_upload_image:37 - Uploading assets/renders/scene.png to Cloudinary...



Blender quit


2025-06-02 22:27:18.800 | SUCCESS  | __main__:render_and_upload_image:47 - Upload successfull blender_renders/scene_1748928438 !!
2025-06-02 22:27:18.801 | INFO     | __main__:plan_and_render:105 - Verifying scene ...
2025-06-02 22:27:18.802 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-02 22:27:29.280 | SUCCESS  | __main__:plan_and_render:107 - Verification score: 0.80
2025-06-02 22:27:29.281 | INFO     | __main__:generate_questions:5 - Generating question ...
2025-06-02 22:27:32.562 | INFO     | __main__:answer_questions:4 - Generating answer ...
2025-06-02 22:27:46.792 | SUCCESS  | __main__:plan_and_render:111 - Question 1: Does the desk1 with dimensions 0.8m (width), 0.6m (depth), and 0.75m (height) positioned at [1,0,0.6] overlap with the chair1 having dimensions 0.5m (width), 0.5m (depth), and 0.8m (height) positioned at [1,0,1] in the rendered image of the office room?
2025-06-02 22:27:46.793 | SUCCESS  | __main__:plan_and_render:112 - Answer 1: Assuming bot

In [59]:
office_rendered_scene

{'asset_id': '53f5a3a8b11c45e1dbe5dd53e766337a',
 'public_id': 'blender_renders/scene_1748928438',
 'version': 1748928438,
 'version_id': '052433688ccf6c9ff8f14c9f3c1c3f0a',
 'signature': 'fe4e7b6c03afa8901b758adec39f3f560090d4d4',
 'width': 1920,
 'height': 1080,
 'format': 'png',
 'resource_type': 'image',
 'created_at': '2025-06-03T05:27:18Z',
 'tags': ['blender', 'render', 'image'],
 'bytes': 1425853,
 'type': 'upload',
 'etag': 'e3cdd3fdb070ddb2f78e6b6fbc629acf',
 'placeholder': False,
 'url': 'http://res.cloudinary.com/cld-store/image/upload/v1748928438/blender_renders/scene_1748928438.png',
 'secure_url': 'https://res.cloudinary.com/cld-store/image/upload/v1748928438/blender_renders/scene_1748928438.png',
 'folder': 'blender_renders',
 'original_filename': 'scene',
 'api_key': '986284565458351'}

In [ ]:
# Creating JSON strings for all results
bedroom_results = {"prompt": bedroom_prompt,
                   "score": bed_score, 
                   "question_1": bed_eval_question1, 
                   "answer_1": bed_eval_answer1, 
                   "question_2": bed_eval_question2, 
                   "answer_2": bed_eval_answer2, 
                   }

bathroom_results = {"prompt": bathroom_prompt,
                    "score": bath_score, 
                   "question_1": bath_eval_question1, 
                   "answer_1": bath_eval_answer1, 
                   "question_2": bath_eval_question2, 
                   "answer_2": bath_eval_answer2, 
                   }

dining_room_results = {"prompt": dining_room_prompt,
                       "score": dining_score, 
                   "question_1": dining_eval_question1, 
                   "answer_1": dining_eval_answer1, 
                   "question_2": dining_eval_question2, 
                   "answer_2": dining_eval_answer2, 
                   }

living_room_results = {"prompt": living_room_prompt,
                       "score": living_score, 
                   "question_1": living_eval_question1, 
                   "answer_1": living_eval_answer1, 
                   "question_2": living_eval_question2, 
                   "answer_2": living_eval_answer2, 
                   }

office_room_results = {"prompt": office_room_prompt,
                       "score": office_score, 
                   "question_1": office_eval_question1, 
                   "answer_1": office_eval_answer1, 
                   "question_2": office_eval_question2, 
                   "answer_2": office_eval_answer2, 
                   }


# storing it in a file
with open("bedroom_resultss.json", "w") as json_file:
    json.dump(bedroom_results, json_file)
print("JSON file created successfully!")

with open("bathroom_resultss.json", "w") as json_file:
    json.dump(bathroom_results, json_file)
print("JSON file created successfully!")

with open("dining_room_resultss.json", "w") as json_file:
    json.dump(dining_room_results, json_file)
print("JSON file created successfully!")

with open("living_room_resultss.json", "w") as json_file:
    json.dump(living_room_results, json_file)
print("JSON file created successfully!")

with open("office_room_resultss.json", "w") as json_file:
    json.dump(office_room_results, json_file)
print("JSON file created successfully!")

JSON file created successfully!
JSON file created successfully!
JSON file created successfully!
JSON file created successfully!
JSON file created successfully!


In [ ]:
# Creating JSON strings for all results
bedroom_results = {"prompt": bedroom_prompt,
                   "score": bed_score, 
                   "question_1": bed_eval_question1, 
                   "answer_1": bed_eval_answer1, 
                   "question_2": bed_eval_question2, 
                   "answer_2": bed_eval_answer2, 
                   }

bathroom_results = {"prompt": bathroom_prompt,
                    "score": bath_score, 
                   "question_1": bath_eval_question1, 
                   "answer_1": bath_eval_answer1, 
                   "question_2": bath_eval_question2, 
                   "answer_2": bath_eval_answer2, 
                   }

dining_room_results = {"prompt": dining_room_prompt,
                       "score": dining_score, 
                   "question_1": dining_eval_question1, 
                   "answer_1": dining_eval_answer1, 
                   "question_2": dining_eval_question2, 
                   "answer_2": dining_eval_answer2, 
                   }

living_room_results = {"prompt": living_room_prompt,
                       "score": living_score, 
                   "question_1": living_eval_question1, 
                   "answer_1": living_eval_answer1, 
                   "question_2": living_eval_question2, 
                   "answer_2": living_eval_answer2, 
                   }

office_room_results = {"prompt": office_room_prompt,
                       "score": office_score, 
                   "question_1": office_eval_question1, 
                   "answer_1": office_eval_answer1, 
                   "question_2": office_eval_question2, 
                   "answer_2": office_eval_answer2, 
                   }


# storing it in a file
with open("bedroom_results.json", "w") as json_file:
    json.dump(bedroom_results, json_file, indent=4)
print("JSON file created successfully!")

with open("bathroom_results.json", "w") as json_file:
    json.dump(bathroom_results, json_file, indent=4)
print("JSON file created successfully!")

with open("dining_room_results.json", "w") as json_file:
    json.dump(dining_room_results, json_file, indent=4)
print("JSON file created successfully!")

with open("living_room_results.json", "w") as json_file:
    json.dump(living_room_results, json_file, indent=4)
print("JSON file created successfully!")

with open("office_room_results.json", "w") as json_file:
    json.dump(office_room_results, json_file, indent=4)
print("JSON file created successfully!")

JSON file created successfully!
JSON file created successfully!
JSON file created successfully!
JSON file created successfully!
JSON file created successfully!
